(basics:first-example-in-ngsolve)=
# A first example in NGSove

This first notebook introduces in few basic steps how to solve a simple PDE problem using NGSolve; each step contains different objects that will be explained in detail in the following notebooks.


To solve the problem one need to consider the following steps:

1. Define the domain $\Omega$, name the boundaries and mesh it.
2. Define the finite element space and reserve space for the solution.
3. Define the linear and bilinear forms and assemble respective matrix and a vector.
4. Solve the linear system. (We may need homogeneization techniques)

## Poisson problem on the unit square

The first problem we are going to solve a problem on the unit square $\Omega = (0,1)^2$, we call the boundaries $\Gamma_b, \Gamma_t, \Gamma_l, \Gamma_r$ the bottom, top, left and right boundaries respectively.

Find $u$ such that
$$
\begin{align*} -\Delta u &= 20\,xy^2 \quad \text{in } \Omega, \\ u &= 0 \quad \text{on } \Gamma_b \cup \Gamma_l,\\ n\cdot \nabla u &= 0 \quad \text{on } \Gamma_r \cup \Gamma_t, \end{align*}
$$
### Step 0: Import the necessary libraries

- `ngsolve` is the library that contains the finite element methods and the solvers.
- `Draw` is a function that allows us to visualize the results in the notebook.

In [21]:
from ngsolve import *
from ngsolve.webgui import Draw # Drawing in Jupyter notebook. More about this later.
import numpy as np
from matplotlib import pyplot as plt

### Step 1: Define the domain





In [22]:
netgen_mesh = unit_square.GenerateMesh(maxh=0.1) # generate a netgen mesh from the OCCGeometry class
mesh = Mesh(netgen_mesh)                         # create a NGSolve mesh class

Draw(mesh); # names are predefined for this case

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

### Step 2: Define the finite element space and solution vector
We need to create the Lagrange finite element space of order 1 on the mesh, to do so me need to pass a mesh and the essential boundary conditions.

The essential boundaries are selected using the `dirichlet` flag. In particular we select the bottom and (united with) the left boundary using the regex `bottom|left`.

In [23]:
fes = H1(mesh, order = 1, dirichlet="bottom|left") # define the finite element space

With the use of a finite element space one can create a `GridFunction` that will be used to store the solution.

For now you can understand the `GridFunction` as a vector that stores the values of the solution at the nodes of the mesh.

In [24]:
gfu = GridFunction(fes) # define the grid function: store a function on the finite element space

# help(gfu) # query the grid function

# print(gfu.vec) # print the vector of the grid function

### Step 3: Define the symbolic forms

Our problem is to find $u $ such that
$$
\begin{align*} \int_{\Omega} \nabla u \cdot \nabla v \,dx &= \int_{\Omega} 20\,xy^2 v \,dx \quad \forall v  \end{align*}
$$
For this specific problem we need to define the bilinear form $a(u,v)$ and the linear form $f(v)$. To write it in a math-like way we make use of the `ProxyFunctions`:
- `u`, trial function.
- `v`, test function.


In [25]:

u = fes.TrialFunction() # define the trial function
v = fes.TestFunction() # define the test function

# u, v = fes.TnT() # one-liner to define the trial and test functions

a = BilinearForm(fes) # define the bilinear form
a += grad(u) * grad(v) * dx
# a += u*v * dx  #?

# print(a.mat)  # print it ... does not work! 

a.Assemble();
from scipy.sparse import csr_matrix
print(csr_matrix(a.mat.CSR()).shape) # print the matrix of the bilinear form

(134, 134)


in the biliear form we have the `dx` that represents the integration over the domain $\Omega$, it is short hand for `DifferentialSymbol(VOL)`.

and `grad` that represents the gradient of the symbolic test/trial functions.

We want to declare the right hand side as follows:
$$
f(v) = \int_{\Omega} 20 \,xy^2\, v \, dx.
$$
To do so we use the `CoefficientFunction`s `x` and `y` that represent the first two coordinate

In [26]:
f = LinearForm(fes)
f += 20 * x * y ** 2 * v * dx
f.Assemble();

print(np.array(f.vec)) # print the vector of the linear form. RHS is zero so far.

[-3.44431912e-21  7.34419573e-05  3.08750000e-02  1.72279853e-03
  2.63972283e-05  6.03161345e-05  6.18548890e-05  6.52109510e-05
  6.57884862e-05  6.31480926e-05  5.74577520e-05  5.05096691e-05
  5.97026061e-05  8.68442771e-04  3.00027324e-03  7.25292286e-03
  1.37593900e-02  2.19704091e-02  3.15836077e-02  4.26793779e-02
  5.48167905e-02  8.17343701e-02  1.08750056e-01  6.02187921e-02
  6.03944013e-02  5.17638743e-02  4.17039878e-02  3.27437134e-02
  2.37495700e-02  1.48250081e-02  7.70786088e-03  1.50585708e-03
  1.15384957e-03  9.94957593e-04  7.20603517e-04  4.34527157e-04
  2.66413893e-04  2.02217103e-04  1.52481574e-04  3.95979393e-05
  6.51971628e-04  8.32455463e-04  9.11099780e-04  9.44115611e-04
  9.13706153e-04  8.43277863e-04  7.14059564e-04  5.57133528e-04
  7.74754863e-04  3.13126915e-03  9.09502296e-03  2.00615149e-02
  3.49108567e-02  5.19849264e-02  7.24376872e-02  9.60426662e-02
  1.27164721e-01  9.33603389e-02  1.12462493e-01  1.06145784e-01
  8.22753629e-02  6.56348

### Step 4: Solve the linear system

So far our problem was defined only symbolically, the only computation that was done was in the creation of the mesh. To actively solve the problem we need to assemble the linear system.

We can now invert the matrix and solve the linear system.

In [27]:
inv = a.mat.Inverse(freedofs=fes.FreeDofs()) # compute the inverse of the matrix

gfu.vec.data = inv * f.vec  # solve the system

What freedofs does is to select the degrees of freedom that are not in the essential boundary conditions. 

In [28]:
Draw(gfu); # draw the solution

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [29]:
np.array(gfu.vec)

array([0.        , 0.        , 2.15499579, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.24787221, 0.50095972,
       0.7606392 , 1.02490383, 1.28872796, 1.54383194, 1.77801897,
       1.97476691, 2.1130173 , 2.13522428, 2.02474847, 1.87164339,
       1.67487488, 1.44315994, 1.1848489 , 0.90620841, 0.6123876 ,
       0.30842807, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.05633485, 0.09658013, 0.12653536, 0.14928282, 0.1643806 ,
       0.17224895, 0.17282005, 0.17074245, 0.19515492, 0.40307994,
       0.6314234 , 0.88654665, 1.15491701, 1.41984884, 1.66765894,
       1.88401006, 2.04912712, 2.03788352, 1.92556378, 1.75425082,
       1.54433848, 1.30723369, 1.04686183, 0.77306509, 0.5057685 ,
       0.24837361, 0.2146583 , 0.20841996, 0.18890727, 0.15456656,
       0.11837536, 0.09866996, 0.08542422, 0.22702445, 0.15923

## Interact with NGSolve


* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python module in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  
* One can use the Netgen GUI within a jupyter notebook: `import netgen.gui`. 
  ```python
  import netgen.gui
  from ngsolve import *

  mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))  # generate a mesh
  Draw(mesh);  # draw the mesh
  ```


In [30]:
# import netgen.gui
# from ngsolve import *

# mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))  # generate a mesh
# Draw(mesh);  # draw the mesh